ASAP CRN Metadata validation 

# Team Voet. ASAP CRN Metadata validation 

17 Oct 2024
Andy Henrie




In [1]:
import pandas as pd
from pathlib import Path
import os, sys

sys.path.append(os.path.abspath((os.path.join(os.getcwd(), 'src/crn_utils'))))

from util import read_CDE, NULL, prep_table, read_meta_table
from validate import validate_table, ReportCollector
from update_schema import v1_to_v2, v2_to_v3_PMDBS, create_upload_medadata_package

%load_ext autoreload
%autoreload 2


Streamlit NOT successfully imported


## CDEs
load the relavent CDEs

In [2]:
schema_version = "v1"
schema_path = Path.home() / "Projects/ASAP/crn-utils/resource/CDE"
CDEv1 = read_CDE(schema_version, local_path=schema_path)
schema_version = "v2.1"
CDEv2 = read_CDE(schema_version, local_path=schema_path)
schema_version = "v3.0"
CDEv3 = read_CDE(schema_version, local_path=schema_path)

metadata_version: ASAP_CDE_v1
https://docs.google.com/spreadsheets/d/1c0z5KvRELdT2AtQAH2Dus8kwAyyLrR0CROhKOjpU4Vc/gviz/tq?tqx=out:csv&sheet=v1
/Users/ergonyc/Projects/ASAP/crn-utils/resource/CDE/ASAP_CDE_v1.csv
read local file
metadata_version: ASAP_CDE_v2.1
https://docs.google.com/spreadsheets/d/1c0z5KvRELdT2AtQAH2Dus8kwAyyLrR0CROhKOjpU4Vc/gviz/tq?tqx=out:csv&sheet=v2.1
/Users/ergonyc/Projects/ASAP/crn-utils/resource/CDE/ASAP_CDE_v2.1.csv
read local file
metadata_version: ASAP_CDE_v3.0
https://docs.google.com/spreadsheets/d/1c0z5KvRELdT2AtQAH2Dus8kwAyyLrR0CROhKOjpU4Vc/gviz/tq?tqx=out:csv&sheet=v3.0
/Users/ergonyc/Projects/ASAP/crn-utils/resource/CDE/ASAP_CDE_v3.0.csv
read local file


## Clean V2 Table
write clean metadata tables according to CDE v2

### Team Voet

In [4]:
## convert 
team = "voet"
collection = "aertslab_rna"
# collection = "multiome_rnaseq"

root_path = Path.home() / ("Projects/ASAP/data/teams")

metadata_path = root_path / f"{team}/{collection}/metadata/"
v3_path = metadata_path / "v3"
og_path = metadata_path / "og"  # this data is a copy of the lee/sn_pmdbs/metadata/v3

v3_meta_tables = ['STUDY', 'PROTOCOL', 'SUBJECT', 'SAMPLE', 'DATA', 'CLINPATH', 'PMDBS', 'CONDITION', 'ASSAY_RNAseq']
og_meta_tables = ['STUDY', 'PROTOCOL', 'SUBJECT', 'SAMPLE', 'DATA', 'CLINPATH', 'PMDBS', 'ASSAY_scRNAseq']

in_tables = [table_name for table_name in og_meta_tables if f"{table_name}.csv" in os.listdir(og_path)]

In [5]:
og_tables = {}
for table_name in in_tables:
    df = read_meta_table(f"{og_path}/{table_name}.csv")
    og_tables[table_name] = df
og_tables.keys() 

dict_keys(['STUDY', 'PROTOCOL', 'SUBJECT', 'SAMPLE', 'DATA', 'CLINPATH', 'ASSAY_scRNAseq'])

In [6]:
# rationalize name
og_tables['STUDY']['team_dataset_id'] = "singlecellmultiome"
og_tables['STUDY']

,ASAP_team_name,ASAP_lab_name,project_name,team_dataset_id,project_dataset,project_description,PI_full_name,PI_email,contributor_names,submitter_name,...,publication_PMID,numbe_samples,sample_types,types_of_samples,DUA_version,PI_ORCHID,PI_google_scholar_id,preprocessing_references,metadata_version_date,alternate_dataset_id
0,TEAM-VOET,Aerts Lab,Understanding inherited and acquired genetic v...,singlecellmultiome,Aerts_singlecellmultiome_RNA,The functional roles of inherited and acquired...,Stein Aerts,stein.aerts@kuleuven.be,"Alexandra Pan?�kov�, Julie De Man, Gert Hulsel...",Gert Hulselmans,...,NA,335,"Anterior Cingulate Cortex, Mid Brain, Motor Co...",late-stage PD and control postmortem brains,NA,0000-0002-8006-0315,NA,NA,"metadata version 1, 20240829",NA


In [12]:
CDE = CDEv3
dfs = {}
for table,df in og_tables.items():
    if table == "ASSAY_scRNAseq":
        table = "ASSAY_RNAseq"

    schema = CDE[CDE['Table'] == table]

    report = ReportCollector(destination="NA")
    full_table, report = validate_table(df.copy(), table, schema, report)
    report.print_log()
    dfs[table] = full_table

🚨⚠️❗ **Missing Required Fields in STUDY: number_samples, metadata_tables**
🚨⚠️❗ **Missing Optional Fields in STUDY: PI_ORCID**
🚨⚠️❗ **7 Fields with empty (NULL) values:**

	- other_funding_source: 1/1 empty rows (REQUIRED)

	- publication_DOI: 1/1 empty rows (REQUIRED)

	- publication_PMID: 1/1 empty rows (REQUIRED)

	- DUA_version: 1/1 empty rows (REQUIRED)

	- PI_google_scholar_id: 1/1 empty rows (OPTIONAL)

	- preprocessing_references: 1/1 empty rows (OPTIONAL)

	- alternate_dataset_id: 1/1 empty rows (OPTIONAL)
No invalid entries found in Enum fields.
🚨⚠️❗ **Extra field in STUDY: numbe_samples**
🚨⚠️❗ **Extra field in STUDY: PI_ORCHID**

All required fields are present in *PROTOCOL* table.
No empty entries (NULL) found .
No invalid entries found in Enum fields.

All required fields are present in *SUBJECT* table.
🚨⚠️❗ **Missing Optional Fields in SUBJECT: primary_diagnosis_text**
🚨⚠️❗ **1 Fields with empty (NULL) values:**

	- race: 284/334 empty rows (REQUIRED)
No invalid entries f

In [13]:
# the fields which nmoved from SUBJECT -> CLINPATH are all empty... so we can just ignore
# CLINPATH[[
#     "AMPPD_id",
# "GP2_id",
# "ethnicity",
# "family_history",
# "last_diagnosis",
# "age_at_onset",
# "age_at_diagnosis",
# "first_motor_symptom",
# "hx_dementia_mci",
# "hx_melanoma",
# "education_level",
# "smoking_status",
# "smoking_years",
# "APOE_e4_status",
# "cognitive_status",
# "time_from_baseline",
# ]]
dfs.keys()

dict_keys(['STUDY', 'PROTOCOL', 'SUBJECT', 'SAMPLE', 'DATA', 'CLINPATH', 'ASSAY_RNAseq'])

In [15]:
STUDY = dfs['STUDY']
PROTOCOL = dfs['PROTOCOL']
SUBJECT = dfs['SUBJECT']
SAMPLE = dfs['SAMPLE']
DATA = dfs['DATA']
CLINPATH = dfs['CLINPATH']
ASSAY_RNAseq = dfs['ASSAY_RNAseq']

STUDY_ = og_tables['STUDY']
PROTOCOL_ = og_tables['PROTOCOL']
SUBJECT_ = og_tables['SUBJECT']
SAMPLE_ = og_tables['SAMPLE']
DATA_ = og_tables['DATA']
CLINPATH_ = og_tables['CLINPATH']
ASSAY_RNAseq_ = og_tables['ASSAY_scRNAseq']


This multiome data is complicated compared to other submissions as data are multiplexed across subjects.  Need to confirm with team where genetic information to assign cells to individual subjects can be found.

Also need to test the fidelity of running this multiome (ARC) assay results along with rna seq only data.  Should be kosher.

Current 'sample_id' encoding is inconsistent between SAMPLE and DATA tables.

In [ ]:
# add 

subject_id 

In [ ]:

PMDBS = dfs['PMDBS']
CONDITION = dfs['CONDITION']

## Validate and clean the metadata tables

In [3]:
metadata_version = "v2"

CDE_df,dtypes_dict = read_ASAP_CDE(metadata_version=metadata_version)
METADATA_VERSION_DATE = f"{metadata_version}_{pd.Timestamp.now().strftime('%Y%m%d')}"


## convert 
data_path = Path.home() / ("Projects/ASAP/team-sulzer")
metadata_path = data_path / "metadata_email"


SUBJECT = read_meta_table(metadata_path / "SUBJECT.csv", dtypes_dict)
CLINPATH = read_meta_table(metadata_path / "CLINPATH.csv", dtypes_dict)
STUDY = read_meta_table(metadata_path / "STUDY.csv", dtypes_dict)
PROTOCOL = read_meta_table(metadata_path / "PROTOCOL.csv", dtypes_dict)
SAMPLE = read_meta_table(metadata_path / "SAMPLE.csv", dtypes_dict)
DATA = read_meta_table(metadata_path / "DATA.csv", dtypes_dict)

# SUBJECT = pd.read_csv(f"{metadata_path}/SUBJECT.csv")
# CLINPATH = pd.read_csv(f"{metadata_path}/CLINPATH.csv")
# STUDY = pd.read_csv(f"{metadata_path}/STUDY.csv")
# PROTOCOL = pd.read_csv(f"{metadata_path}/PROTOCOL.csv")
# SAMPLE = pd.read_csv(f"{metadata_path}/SAMPLE.csv")
# DATA = pd.read_csv(f"{metadata_path}/DATA.csv")


read url


In [4]:

STUDY = prep_table(STUDY, CDE_df)
# update metadata_version_date
STUDY['metadata_version_date'] = METADATA_VERSION_DATE

In [5]:
report = ReportCollector(destination="")
validate_table(STUDY, "STUDY", CDE_df, report)
report.print_log()

All required fields are present in *STUDY* table.
No empty entries (Nan) found in _Required_ fields.
No empty entries (Nan) found in _Optional_ fields.
## Enum fields have valid values in STUDY. 🥳



In [6]:
# fix the column order
PROTOCOL = prep_table(PROTOCOL, CDE_df)
report = ReportCollector(destination="")
validate_table(PROTOCOL, "PROTOCOL", CDE_df, report)
report.print_log()



All required fields are present in *PROTOCOL* table.
No empty entries (Nan) found in _Required_ fields.
No empty entries (Nan) found in _Optional_ fields.
## Enum fields have valid values in PROTOCOL. 🥳



In [7]:

SUBJECT = prep_table(SUBJECT, CDE_df)



SUBJECT['sex'].replace({"M":"Male", "F":"Female"}, inplace=True)


In [8]:

SUBJECT['hx_dementia_mci'].replace("Unknown", pd.NA, inplace=True)
SUBJECT['hx_melanoma'].replace("Unknown", pd.NA, inplace=True)
SUBJECT['education_level'].replace("Unknown", pd.NA, inplace=True)
SUBJECT['cognitive_status'].replace("Unknown", pd.NA, inplace=True)
SUBJECT['primary_diagnosis'].replace("Unknown", pd.NA, inplace=True)


In [9]:
report = ReportCollector(destination="")
validate_table(SUBJECT, "SUBJECT", CDE_df, report)
report.print_log()

All required fields are present in *SUBJECT* table.
No empty entries (Nan) found in _Required_ fields.
No empty entries (Nan) found in _Optional_ fields.
## Enums
🚨⚠️❗ **Invalid entries**
- _*hx_dementia_mci*_:  invalid values 💩'<NA>'
    - valid ➡️ 'Yes', 'No'
- _*hx_melanoma*_:  invalid values 💩'<NA>'
    - valid ➡️ 'Yes', 'No'
- _*education_level*_:  invalid values 💩'<NA>'
    - valid ➡️ 'High School', 'High School/GED', 'Some college without degree', 'Associate degree college', 'Bachelor's degree', 'Master's degree', 'Professional or doctoral degree', 'Refuse', 'Other'
- _*cognitive_status*_:  invalid values 💩'<NA>'
    - valid ➡️ 'Normal', 'MCI', 'Dementia'
- _*primary_diagnosis*_:  invalid values 💩'<NA>'
    - valid ➡️ 'Healthy Control', 'Idiopathic PD', 'Alzheimer’s disease', 'Frontotemporal dementia', 'Corticobasal syndrome', 'Dementia with Lewy bodies', 'Dopa-responsive dystonia', 'Essential tremor', 'Hemiparkinson/hemiatrophy syndrome', 'Juvenile autosomal recessive parkinson

In [10]:

SAMPLE["organism_ontology_term_id"] = "NCBITaxon:9606"
SAMPLE['molecular_source'].replace("RNA", "PolyA RNA", inplace=True)
SAMPLE['region_level_2'].replace('Substantia Nigra', 'Substantia nigra', inplace=True)

In [11]:
batch_mapper = {"Cingulate Cortex":"cing1", "Substantia Nigra": "sn1"}
SAMPLE['batch']=SAMPLE['brain_region'].map(batch_mapper)

In [12]:
SAMPLE = prep_table(SAMPLE, CDE_df)
report = ReportCollector(destination="")
validate_table(SAMPLE, "SAMPLE", CDE_df, report)
report.print_log()

All required fields are present in *SAMPLE* table.
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- RIN: 28/46 empty rows

	- source_RIN: 46/46 empty rows

	- input_cell_count: 28/46 empty rows
🚨⚠️❗ **Optional Fields with Empty (nan) values:**

	- DV200: 46/46 empty rows

	- pm_PH: 46/46 empty rows
## Enums
🚨⚠️❗ **Invalid entries**
- _*hemisphere*_:  invalid values 💩'<NA>'
    - valid ➡️ 'Right', 'Left', 'Unknown'



In [13]:
# CLINPATH.fillna("Nan")
report = ReportCollector(destination="")
validate_table(CLINPATH, "CLINPATH", CDE_df, report)
report.print_log()



All required fields are present in *CLINPATH* table.
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- path_year_death: 46/46 empty rows

	- brain_weight: 46/46 empty rows
No empty entries (Nan) found in _Optional_ fields.
## Enums
🚨⚠️❗ **Invalid entries**
- _*path_braak_nft*_:  invalid values 💩'<NA>'
    - valid ➡️ '0', 'I', 'II', 'III', 'IV', 'V', 'VI', 'I/II', 'III/IV', 'IV/V', 'V/VI'
- _*path_cerad*_:  invalid values 💩'<NA>'
    - valid ➡️ 'None', 'Sparse', 'Moderate', 'Frequent'
- _*path_thal*_:  invalid values 💩'<NA>'
    - valid ➡️ '0', '1', '2', '3', '4', '5', '1/2', '3', '4/5'
- _*known_pathogenic_mutation*_:  invalid values 💩'<NA>'
    - valid ➡️ 'None', 'Present', 'Unknown'
- _*path_mckeith*_:  invalid values 💩'<NA>'
    - valid ➡️ 'Neocortical', 'Limbic (transitional)', 'Brainstem', 'Amygdala Predominant', 'Olfactory bulb only', 'Limbic, transitional (brainstem and limbic involvement)', 'Diffuse, neocortical (brainstem, limbic and neocortical involvement)', 'Olfactory B

In [14]:
CLINPATH = prep_table(CLINPATH, CDE_df)
report = ReportCollector(destination="")
validate_table(CLINPATH, "CLINPATH", CDE_df, report)
report.print_log()


All required fields are present in *CLINPATH* table.
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- path_year_death: 46/46 empty rows

	- brain_weight: 46/46 empty rows
No empty entries (Nan) found in _Optional_ fields.
## Enums
🚨⚠️❗ **Invalid entries**
- _*path_braak_nft*_:  invalid values 💩'<NA>'
    - valid ➡️ '0', 'I', 'II', 'III', 'IV', 'V', 'VI', 'I/II', 'III/IV', 'IV/V', 'V/VI'
- _*path_cerad*_:  invalid values 💩'<NA>'
    - valid ➡️ 'None', 'Sparse', 'Moderate', 'Frequent'
- _*path_thal*_:  invalid values 💩'<NA>'
    - valid ➡️ '0', '1', '2', '3', '4', '5', '1/2', '3', '4/5'
- _*known_pathogenic_mutation*_:  invalid values 💩'<NA>'
    - valid ➡️ 'None', 'Present', 'Unknown'
- _*path_mckeith*_:  invalid values 💩'<NA>'
    - valid ➡️ 'Neocortical', 'Limbic (transitional)', 'Brainstem', 'Amygdala Predominant', 'Olfactory bulb only', 'Limbic, transitional (brainstem and limbic involvement)', 'Diffuse, neocortical (brainstem, limbic and neocortical involvement)', 'Olfactory B

In [15]:
DATA.head()

,sample_id,replicate,replicate_count,repeated_sample,batch,file_type,file_name,file_description,file_MD5,technology,omic,adjustment,content,time,header,annotation,configuration_file
0,T-362,rep1,1,0,1,fastq,<NA>,<NA>,<NA>,sN,RNA,Raw,Counts,NaN,<NA>,<NA>,<NA>
1,T-308,rep1,1,0,1,fastq,<NA>,<NA>,<NA>,sN,RNA,Raw,Counts,NaN,<NA>,<NA>,<NA>
2,T3860,rep1,1,0,1,fastq,<NA>,<NA>,<NA>,sN,RNA,Raw,Counts,NaN,<NA>,<NA>,<NA>
3,T290,rep1,1,0,1,fastq,<NA>,<NA>,<NA>,sN,RNA,Raw,Counts,NaN,<NA>,<NA>,<NA>
4,T-233,rep1,1,0,1,fastq,<NA>,<NA>,<NA>,sN,RNA,Raw,Counts,NaN,<NA>,<NA>,<NA>


## 🚨⚠️❗ EXTREME CURATION HERE 🚨⚠️❗
The codeing by Team-Sulzer is inconsistent.  However I am able to infer the following:

TODO:  Define `sample_id`s from `subject_id`, and create `sample_id` as `source_subject_id`


`source_subject_id`s seem to be unique and correct.  Although the CLINPATH.duration_pmi are different for `source_subject_id`="T-2465", and some demographic info is missing for the cingulate sample, these seem to be two samples from the same subject.  

Hence:  We will assume `source_subject_id` is ground truth.   Redefine `sample_id` from the current `subject_id` (which matches file naming conventsions), and  create a _new_ `subject_id` from the `source_subject_id` by prepending "SUL_".  Finally, `source_sample_id` will be set to pd.NA as is currently just a copy of `source_subject_id`

### fix SAMPLE table first

In [16]:
# simple copy subject_id to sample_id
SAMPLE['sample_id'] = SAMPLE['subject_id']


In [17]:

# then make a subject_id by copying souce_sample_id
SAMPLE['subject_id'] = "SUL_" + SAMPLE['source_sample_id']

# finally set source_sample_id to pd.NA
SAMPLE['source_sample_id'] = pd.NA

### now SUBJECT table 

In [18]:
SUBJECT['subject_id'] = "SUL_" + SUBJECT['source_subject_id']

### and CLINPATH table is already coded as it should be



## DATA table

The DATA.csv was not filled out.  So we will construct it by joining the manifest from the source of data transfer for Team Sulzer.

### read_file_list
Helper function to read sulzer manifest 

In [19]:

cingulate_files = Path.home() / "Projects/ASAP/team-sulzer/cingulate_manifest.txt"
nigra_files = Path.home() / "Projects/ASAP/team-sulzer/nigra_manifest.txt"

In [20]:
# tmp2 = pd.read_csv("sulzer_fastqs.txt", delimiter=" ", header=None, names=['file_name'])
def read_gz_file_list(file_nm):
    # Read the text file
    with open(file_nm, 'r') as file:
        lines = [line.strip() for line in file]

    # Create a DataFrame
    df = pd.DataFrame(lines, columns=['file_name'])

    df['parts'] = df['file_name'].apply(lambda x: x.lstrip("gs://").split("/"))
    df['source'] = df['parts'].apply(lambda x: x[1].lstrip("fastqs_") )
    df['name'] = df['parts'].apply(lambda x: x[-1])

    df['name_parts'] = df['name'].apply(lambda x: x.rstrip('.fastq.gz').split("_"))
    df['sample_id'] = df['name_parts'].apply(lambda x: x[0])
    df['lane_id'] = df['name_parts'].apply(lambda x: x[2])
    df['file_type'] = df['name_parts'].apply(lambda x: x[3])
    df.drop(columns=['parts','name_parts'], inplace=True)
    return df

# tmp2 = pd.read_csv("sulzer_fastqs.txt", delimiter=" ", header=None, names=['file_name'])
def read_file_list_sulzer(file_nm):
    # Read the text file
    keep_ends = ("fastq.gz", "bam", "tar")
    with open(file_nm, 'r') as file:
        lines = [line.strip() for line in file if line.strip().endswith(keep_ends)]

    # Create a DataFrame
    df = pd.DataFrame(lines, columns=['file_name'])

    df['parts'] = df['file_name'].apply(lambda x: x.split("/"))
    # df['source'] = df['parts'].apply(lambda x: x[5].lstrip("fastqs_") )
    df['file_name'] = df['parts'].apply(lambda x: x[-1])
    df['sample_id'] = df['parts'].apply(lambda x: x[6] if x[-1].endswith(("fastq.gz","tar")) else x[7])
    df['file_type'] = df['parts'].apply(lambda x: x[7] if x[7].strip() in ["fastq","analysis"] else "bam")
    # df['tmp1'] = df['parts'].apply(lambda x: x[7] if len(x) > 7 else pd.NA)

    # df['name_parts'] = df['name'].apply(lambda x: x.rstrip('.fastq.gz').split("_"))
    # df['sample_id'] = df['name_parts'].apply(lambda x: x[0])
    # df['lane_id'] = df['name_parts'].apply(lambda x: x[2])
    # df['file_type'] = df['name_parts'].apply(lambda x: x[3])
    df.drop(columns=['parts'], inplace=True)
    return df




In [21]:

c_fastqs = read_file_list_sulzer(cingulate_files)
c_fastqs['src'] = "cingulate"
n_fastqs = read_file_list_sulzer(nigra_files)
n_fastqs['src'] = "nigra"

from_source = pd.concat([c_fastqs, n_fastqs])

In [22]:
n_fastqs

,file_name,sample_id,file_type,src
0,OJ1_cellranger_count_outs.tar,OJ1,analysis,nigra
1,OJ1_S1_L001_I1_001.fastq.gz,OJ1,fastq,nigra
2,OJ1_S1_L001_I2_001.fastq.gz,OJ1,fastq,nigra
3,OJ1_S1_L001_R1_001.fastq.gz,OJ1,fastq,nigra
4,OJ1_S1_L001_R2_001.fastq.gz,OJ1,fastq,nigra
...,...,...,...,...
454,OJ9_S1_L003_R2_001.fastq.gz,OJ9,fastq,nigra
455,OJ9_S1_L004_I1_001.fastq.gz,OJ9,fastq,nigra
456,OJ9_S1_L004_I2_001.fastq.gz,OJ9,fastq,nigra
457,OJ9_S1_L004_R1_001.fastq.gz,OJ9,fastq,nigra


In [23]:
## now make DATA from scratch to ready to join into
# Start with sample_id
DATA_ = read_meta_table(metadata_path / "DATA.csv", dtypes_dict)

DATA = SAMPLE[["sample_id", "replicate", "replicate_count","repeated_sample","batch"]].copy()

DATA.head()



,sample_id,replicate,replicate_count,repeated_sample,batch
0,OF001,Rep1,1,0,cing1
1,OF009,Rep1,1,0,cing1
2,OF016,Rep1,1,0,cing1
3,OF005,Rep1,1,0,cing1
4,OF013,Rep1,1,0,cing1


In [24]:
DATA = pd.merge(DATA, from_source, on='sample_id', how='left')
DATA.tail()

,sample_id,replicate,replicate_count,repeated_sample,batch,file_name,file_type,src
707,OJ26,Rep1,1,0,sn1,OJ26_S1_L003_R2_001.fastq.gz,fastq,nigra
708,OJ26,Rep1,1,0,sn1,OJ26_S1_L004_I1_001.fastq.gz,fastq,nigra
709,OJ26,Rep1,1,0,sn1,OJ26_S1_L004_I2_001.fastq.gz,fastq,nigra
710,OJ26,Rep1,1,0,sn1,OJ26_S1_L004_R1_001.fastq.gz,fastq,nigra
711,OJ26,Rep1,1,0,sn1,OJ26_S1_L004_R2_001.fastq.gz,fastq,nigra


In [25]:
DATA['omic'] = 'RNA'
DATA['adjustments'] = 'Raw'
DATA['technology'] = "SN"
DATA['file_description']= DATA['file_type'].map({"bam": "Binary aligned sequence",
                                                "fastq": "gzip fastq",
                                                "analysis": "cellranger counts tar archive"})
DATA['file_description']= DATA['file_type'].map({"bam": "Reads",
                                                "fastq": "Reads",
                                                "analysis": "Counts"})

# now fix the file_type
DATA['file_type'] = DATA['file_type'].replace({'fastq':'fastq', "bam":'Per sample raw file', "analysis":'Per sample processed file'})
DATA['content'] = 'Counts'



get "md5s"

In [26]:


cingulate_md5s = Path.home() / "Projects/ASAP/team-sulzer/cingulate_fastq_md5.log"
nigra_md5s = Path.home() / "Projects/ASAP/team-sulzer/nigra_fastq_md5.log"

In [27]:
# cingulate
cingulate_hashs = extract_hashes_from_gsutil(cingulate_md5s)

# nigra
nigra_hashs = extract_hashes_from_gsutil(nigra_md5s)

# combine cingulate and nigra
source_file_crc = cingulate_hashs[0] | nigra_hashs[0]
source_file_md5 = cingulate_hashs[1] | nigra_hashs[1]


In [28]:
DATA['file_MD5'] = DATA['file_name'].map(source_file_md5)

In [29]:

DATA = reorder_table_to_CDE(DATA,"DATA", CDE_df)
DATA.head()



added empty column adjustment to DATA table
added empty column time to DATA table
added empty column header to DATA table
added empty column annotation to DATA table
added empty column configuration_file to DATA table


,sample_id,replicate,replicate_count,repeated_sample,batch,file_type,file_name,file_description,file_MD5,technology,omic,adjustment,content,time,header,annotation,configuration_file
0,OF001,Rep1,1,0,cing1,Per sample raw file,possorted_genome_bam.bam,Reads,NaN,SN,RNA,<NA>,Counts,<NA>,<NA>,<NA>,<NA>
1,OF001,Rep1,1,0,cing1,Per sample processed file,OF001_cellranger_count_outs.tar,Counts,NaN,SN,RNA,<NA>,Counts,<NA>,<NA>,<NA>,<NA>
2,OF001,Rep1,1,0,cing1,fastq,OF001_S1_L001_I1_001.fastq.gz,Reads,74a74c8929b01a6da7e79b83f91a6607,SN,RNA,<NA>,Counts,<NA>,<NA>,<NA>,<NA>
3,OF001,Rep1,1,0,cing1,fastq,OF001_S1_L001_I2_001.fastq.gz,Reads,644d95728b50f1ff9db7becfa5b5c394,SN,RNA,<NA>,Counts,<NA>,<NA>,<NA>,<NA>
4,OF001,Rep1,1,0,cing1,fastq,OF001_S1_L001_R1_001.fastq.gz,Reads,129dfb6fe6115966e2cf2af5e3c82bdf,SN,RNA,<NA>,Counts,<NA>,<NA>,<NA>,<NA>


In [30]:

report = ReportCollector(destination="")
validate_table(DATA, "DATA", CDE_df, report)
report.print_log()


All required fields are present in *DATA* table.
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- time: 712/712 empty rows
No empty entries (Nan) found in _Optional_ fields.
## Enums
🚨⚠️❗ **Invalid entries**
- _*file_type*_:  invalid values 💩'nan'
    - valid ➡️ 'fastq', 'Per sample raw file', 'Per sample processed file', 'Combined analysis files', 'annData', 'vHDF', 'plink2', 'VCF', 'csv'
- _*adjustment*_:  invalid values 💩'<NA>'
    - valid ➡️ 'Raw', 'Processed'



# No data found for `OJ8`!!!

# load manifests and md5s

Separate manifests for cingulate and nigra to make it easier for Team Sulzer to collect.


In [31]:
SAMPLE[SAMPLE['sample_id']=='OJ8']
empty_sample = SAMPLE['sample_id']=='OJ8'
print(SAMPLE.shape)
SAMPLE = SAMPLE[~empty_sample]
SAMPLE.shape

(46, 33)


(45, 33)

In [32]:
DATA[DATA['sample_id']=='OJ8']
empty_sample = DATA['sample_id']=='OJ8'
print(DATA.shape)
DATA = DATA[~empty_sample]
DATA.shape

(712, 17)


(711, 17)

## check remote files



In [31]:
!gcloud auth activate-service-account --key-file=/Users/ergonyc/Projects/ASAP/sulzer-credentials.json 


Activated service account credentials for: [raw-admin-sulzer@dnastack-asap-parkinsons.iam.gserviceaccount.com]


In [29]:

!gsutil -u dnastack-asap-parkinsons ls -al "gs://asap-raw-data-team-sulzer/fastqs_cingulate_updated/**/*.gz"


   4249181  2023-12-01T21:26:39Z  gs://asap-raw-data-team-sulzer/fastqs_cingulate_updated/OF001/analysis/201123_OSAMA_FATIMA_2_HUMAN_10X-OF001-cellranger-count-default/OF001_cellranger_count_info.tar.gz#1701465999745881  metageneration=1
 752394240  2023-12-02T00:16:13Z  gs://asap-raw-data-team-sulzer/fastqs_cingulate_updated/OF001/fastq/SN_3PGEX_10X/210210_A00521_0199_BHNMK3DSXY/OF001_S1_L001_I1_001.fastq.gz#1701476173757016  metageneration=1
 757577390  2023-12-02T00:24:37Z  gs://asap-raw-data-team-sulzer/fastqs_cingulate_updated/OF001/fastq/SN_3PGEX_10X/210210_A00521_0199_BHNMK3DSXY/OF001_S1_L001_I2_001.fastq.gz#1701476677261783  metageneration=1
7398635734  2023-12-02T18:37:54Z  gs://asap-raw-data-team-sulzer/fastqs_cingulate_updated/OF001/fastq/SN_3PGEX_10X/210210_A00521_0199_BHNMK3DSXY/OF001_S1_L001_R1_001.fastq.gz#1701542274674731  metageneration=1
5587519431  2023-12-02T15:13:17Z  gs://asap-raw-data-team-sulzer/fastqs_cingulate_updated/OF001/fastq/SN_3PGEX_10X/210210_A00521_019

## fix missing file_name, file_description, file_MD5 

Collect "destination" or bucket md5s and manifest

In [74]:

!gsutil -u dnastack-asap-parkinsons ls "gs://asap-raw-data-team-sulzer/**/*fastq.gz" > sulzer_bucket_fasqs.txt

In [76]:
!gsutil -u dnastack-asap-parkinsons hash -h "gs://asap-raw-data-team-sulzer/**/*.gz" > sulzer_bucket_hexhash.log


using the module's C extension, so checksumming will run very slowly. For help
installing the extension, please see "gsutil help crcmod".



In [96]:
# bucket_fastqs = pd.read_csv("sulzer_bucket_fasqs.txt", delimiter="  ", header=None, names=['file_name'])
# bucket_fastqs

In [33]:
bucket_files_md5 = extract_md5_from_details2("sulzer_bucket_hexhash.log")


In [34]:
tmp = pd.DataFrame.from_dict(bucket_files_md5, orient="index",columns=['md5'])
tmp['file'] = tmp.index
tmp = tmp.sort_values(by='file').reset_index()

tmp


,index,md5,file
0,OF001_S1_L001_I1_001.fastq.gz,74a74c8929b01a6da7e79b83f91a6607,OF001_S1_L001_I1_001.fastq.gz
1,OF001_S1_L001_I2_001.fastq.gz,644d95728b50f1ff9db7becfa5b5c394,OF001_S1_L001_I2_001.fastq.gz
2,OF001_S1_L001_R1_001.fastq.gz,129dfb6fe6115966e2cf2af5e3c82bdf,OF001_S1_L001_R1_001.fastq.gz
3,OF001_S1_L001_R2_001.fastq.gz,dbdec2b83d4564e67284a51c8d1f5dc2,OF001_S1_L001_R2_001.fastq.gz
4,OF001_S1_L002_I1_001.fastq.gz,50d1c4be0b4c65310fac646fd676f3c4,OF001_S1_L002_I1_001.fastq.gz
...,...,...,...
724,OJ9_S1_L004_I1_001.fastq.gz,2dba0b38f76ba6490260266e02689392,OJ9_S1_L004_I1_001.fastq.gz
725,OJ9_S1_L004_I2_001.fastq.gz,ed6433715131d4ea289a5644bb89ce0b,OJ9_S1_L004_I2_001.fastq.gz
726,OJ9_S1_L004_R1_001.fastq.gz,a6ec79d470af33aab7e72c6703ea26fa,OJ9_S1_L004_R1_001.fastq.gz
727,OJ9_S1_L004_R2_001.fastq.gz,a9cb6810affb5ae38115d980048c31f9,OJ9_S1_L004_R2_001.fastq.gz


## Only check the fastq.gz files.  don't have md5s for the others

In [35]:
DATA_ = DATA[DATA['file_type']=="fastq"]

In [36]:



checksum = DATA_[['file_name','file_MD5']]
checksum['check1'] = checksum['file_MD5'].str.strip()
checksum['check2'] = checksum['file_name'].map(bucket_files_md5)



/var/folders/0z/hvx6j8cn6yj3bqbyq6rtsxm40000gn/T/ipykernel_40441/2423319654.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checksum['check1'] = checksum['file_MD5'].str.strip()
/var/folders/0z/hvx6j8cn6yj3bqbyq6rtsxm40000gn/T/ipykernel_40441/2423319654.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checksum['check2'] = checksum['file_name'].map(bucket_files_md5)


In [37]:
checksum[checksum.check1 != checksum.check2].file_name.to_list()


[]

In [38]:
checksum.head()

,file_name,file_MD5,check1,check2
2,OF001_S1_L001_I1_001.fastq.gz,74a74c8929b01a6da7e79b83f91a6607,74a74c8929b01a6da7e79b83f91a6607,74a74c8929b01a6da7e79b83f91a6607
3,OF001_S1_L001_I2_001.fastq.gz,644d95728b50f1ff9db7becfa5b5c394,644d95728b50f1ff9db7becfa5b5c394,644d95728b50f1ff9db7becfa5b5c394
4,OF001_S1_L001_R1_001.fastq.gz,129dfb6fe6115966e2cf2af5e3c82bdf,129dfb6fe6115966e2cf2af5e3c82bdf,129dfb6fe6115966e2cf2af5e3c82bdf
5,OF001_S1_L001_R2_001.fastq.gz,dbdec2b83d4564e67284a51c8d1f5dc2,dbdec2b83d4564e67284a51c8d1f5dc2,dbdec2b83d4564e67284a51c8d1f5dc2
6,OF001_S1_L002_I1_001.fastq.gz,50d1c4be0b4c65310fac646fd676f3c4,50d1c4be0b4c65310fac646fd676f3c4,50d1c4be0b4c65310fac646fd676f3c4


## Sanitize NULLs

In [41]:

CLINPATH = sanitize(CLINPATH)
PROTOCOL = sanitize(PROTOCOL)
SUBJECT = sanitize(SUBJECT)
STUDY = sanitize(STUDY)
DATA = sanitize(DATA)
SAMPLE = sanitize(SAMPLE)

## Export to clean

In [42]:

export_root = Path.cwd() / "clean/team-Sulzer"
if not export_root.exists():
    export_root.mkdir(parents=True, exist_ok=True)

STUDY.to_csv( export_root / "STUDY.csv")
PROTOCOL.to_csv(export_root / "PROTOCOL.csv")
SAMPLE.to_csv(export_root / "SAMPLE.csv")
SUBJECT.to_csv(export_root / "SUBJECT.csv")
CLINPATH.to_csv(export_root / "CLINPATH.csv")
DATA.to_csv(export_root / "DATA.csv")


## transfer metadata Sulzer 

In [43]:
# Sulzer
!gcloud auth activate-service-account --key-file=/Users/ergonyc/Projects/ASAP/sulzer-credentials.json 


Activated service account credentials for: [raw-admin-sulzer@dnastack-asap-parkinsons.iam.gserviceaccount.com]


In [44]:

!gsutil -u dnastack-asap-parkinsons cp "./clean/team-sulzer/*.csv"  "gs://asap-raw-data-team-sulzer/metadata/v2/"

Copying file://./clean/team-sulzer/CLINPATH.csv [Content-Type=text/csv]...
Copying file://./clean/team-sulzer/SUBJECT.csv [Content-Type=text/csv]...       
Copying file://./clean/team-sulzer/SAMPLE.csv [Content-Type=text/csv]...        
Copying file://./clean/team-sulzer/DATA.csv [Content-Type=text/csv]...          
- [4 files][107.8 KiB/107.8 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://./clean/team-sulzer/STUDY.csv [Content-Type=text/csv]...
Copying file://./clean/team-sulzer/PROTOCOL.csv [Content-Type=text/csv]...      
\ [6 files][109.1 KiB/109.1 KiB]                                                
Operation completed over 6 objects/109.1 KiB.                                    


### copy metadata to workflow-dev bucket

First copy each set of metadata locally ...

In [65]:
!gcloud auth activate-service-account --key-file=/Users/ergonyc/Projects/ASAP/wf-credentials.json

Activated service account credentials for: [admin-workflow-dev@dnastack-asap-parkinsons.iam.gserviceaccount.com]


In [66]:
!gsutil cp "./clean/team-Sulzer/*.csv" "gs://asap-workflow-dev/metadata/v2/sulzer"

Copying file://./clean/team-Sulzer/CLINPATH.csv [Content-Type=text/csv]...
Copying file://./clean/team-Sulzer/SUBJECT.csv [Content-Type=text/csv]...       
Copying file://./clean/team-Sulzer/SAMPLE.csv [Content-Type=text/csv]...        
Copying file://./clean/team-Sulzer/DATA.csv [Content-Type=text/csv]...          
- [4 files][112.1 KiB/112.1 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://./clean/team-Sulzer/STUDY.csv [Content-Type=text/csv]...
Copying file://./clean/team-Sulzer/PROTOCOL.csv [Content-Type=text/csv]...      
\ [6 files][113.5 KiB/113.5 KiB]                                                
Operation completed over 6 objects/113.5 KiB.                                    


In [67]:
!gsutil ls -al "gs://asap-workflow-dev/metadata/v2"

                                 gs://asap-workflow-dev/metadata/v2/hafler/
                                 gs://asap-workflow-dev/metadata/v2/hardy/
                                 gs://asap-workflow-dev/metadata/v2/jakobsson/
                                 gs://asap-workflow-dev/metadata/v2/lee/
                                 gs://asap-workflow-dev/metadata/v2/scherzer/
                                 gs://asap-workflow-dev/metadata/v2/sulzer/


In [312]:

# move to cannonical location
!gsutil -u dnastack-asap-parkinsons cp -r "gs://asap-raw-data-team-sulzer/fastqs_cingulate_updated" "gs://asap-raw-data-team-sulzer/fastqs/cingulate"


Copying gs://asap-raw-data-team-sulzer/fastqs_cingulate_updated/OF001/analysis/201123_OSAMA_FATIMA_2_HUMAN_10X-OF001-cellranger-count-default/OF001_cellranger_count_info.tar.gz [Content-Type=application/x-tar]...
Copying gs://asap-raw-data-team-sulzer/fastqs_cingulate_updated/OF001/analysis/201123_OSAMA_FATIMA_2_HUMAN_10X-OF001-cellranger-count-default/OF001_cellranger_count_outs.tar [Content-Type=application/x-tar]...
Copying gs://asap-raw-data-team-sulzer/fastqs_cingulate_updated/OF001/analysis/201123_OSAMA_FATIMA_2_HUMAN_10X-OF001-cellranger-count-default/README.txt [Content-Type=text/plain]...
Copying gs://asap-raw-data-team-sulzer/fastqs_cingulate_updated/OF001/analysis/201123_OSAMA_FATIMA_2_HUMAN_10X-OF001-cellranger-count-default/index.html [Content-Type=text/html]...
- [4 files][  1.2 GiB/  1.2 GiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Pl

In [317]:

# move to cannonical location
!gsutil -u dnastack-asap-parkinsons mv -r "gs://asap-raw-data-team-sulzer/fastqs_nigra" "gs://asap-raw-data-team-sulzer/fastqs/nigra"


Copying gs://asap-raw-data-team-sulzer/fastqs_nigra/OJ1/analysis/220224_OSAMA_JUNCHENG_8_HUMAN_10X-OJ1-cellranger-count-default/OJ1_cellranger_count_info.tar.gz [Content-Type=application/x-tar]...
Removing gs://asap-raw-data-team-sulzer/fastqs_nigra/OJ1/analysis/220224_OSAMA_JUNCHENG_8_HUMAN_10X-OJ1-cellranger-count-default/OJ1_cellranger_count_info.tar.gz...
Copying gs://asap-raw-data-team-sulzer/fastqs_nigra/OJ1/analysis/220224_OSAMA_JUNCHENG_8_HUMAN_10X-OJ1-cellranger-count-default/OJ1_cellranger_count_outs.tar [Content-Type=application/x-tar]...
Removing gs://asap-raw-data-team-sulzer/fastqs_nigra/OJ1/analysis/220224_OSAMA_JUNCHENG_8_HUMAN_10X-OJ1-cellranger-count-default/OJ1_cellranger_count_outs.tar...
Copying gs://asap-raw-data-team-sulzer/fastqs_nigra/OJ1/analysis/220224_OSAMA_JUNCHENG_8_HUMAN_10X-OJ1-cellranger-count-default/README.txt [Content-Type=text/plain]...
Removing gs://asap-raw-data-team-sulzer/fastqs_nigra/OJ1/analysis/220224_OSAMA_JUNCHENG_8_HUMAN_10X-OJ1-cellranger

In [350]:

!gsutil -u dnastack-asap-parkinsons mv "gs://asap-raw-data-team-sulzer/fastqs/cingulate/**/*.fastq.gz" "gs://asap-raw-data-team-sulzer/fastqs/cingulate/"


Copying gs://asap-raw-data-team-sulzer/fastqs/cingulate/OF001/fastq/SN_3PGEX_10X/210210_A00521_0199_BHNMK3DSXY/OF001_S1_L001_I1_001.fastq.gz [Content-Type=application/octet-stream]...
Removing gs://asap-raw-data-team-sulzer/fastqs/cingulate/OF001/fastq/SN_3PGEX_10X/210210_A00521_0199_BHNMK3DSXY/OF001_S1_L001_I1_001.fastq.gz...
Copying gs://asap-raw-data-team-sulzer/fastqs/cingulate/OF001/fastq/SN_3PGEX_10X/210210_A00521_0199_BHNMK3DSXY/OF001_S1_L001_I2_001.fastq.gz [Content-Type=application/octet-stream]...
Removing gs://asap-raw-data-team-sulzer/fastqs/cingulate/OF001/fastq/SN_3PGEX_10X/210210_A00521_0199_BHNMK3DSXY/OF001_S1_L001_I2_001.fastq.gz...
Copying gs://asap-raw-data-team-sulzer/fastqs/cingulate/OF001/fastq/SN_3PGEX_10X/210210_A00521_0199_BHNMK3DSXY/OF001_S1_L001_R1_001.fastq.gz [Content-Type=application/octet-stream]...
Removing gs://asap-raw-data-team-sulzer/fastqs/cingulate/OF001/fastq/SN_3PGEX_10X/210210_A00521_0199_BHNMK3DSXY/OF001_S1_L001_R1_001.fastq.gz...
Copying gs://

In [353]:
!gsutil -u dnastack-asap-parkinsons ls -al "gs://asap-raw-data-team-sulzer/fastqs/nigra"


 342559148  2023-12-06T00:19:01Z  gs://asap-raw-data-team-sulzer/fastqs/nigra/OJ10_S1_L001_I1_001.fastq.gz#1701821941986375  metageneration=1
 359917684  2023-12-06T00:19:02Z  gs://asap-raw-data-team-sulzer/fastqs/nigra/OJ10_S1_L001_I2_001.fastq.gz#1701821942302756  metageneration=1
3431481573  2023-12-06T00:19:02Z  gs://asap-raw-data-team-sulzer/fastqs/nigra/OJ10_S1_L001_R1_001.fastq.gz#1701821942601943  metageneration=1
2572117362  2023-12-06T00:19:02Z  gs://asap-raw-data-team-sulzer/fastqs/nigra/OJ10_S1_L001_R2_001.fastq.gz#1701821942905242  metageneration=1
 341079281  2023-12-06T00:19:03Z  gs://asap-raw-data-team-sulzer/fastqs/nigra/OJ10_S1_L002_I1_001.fastq.gz#1701821943223278  metageneration=1
 359720996  2023-12-06T00:19:03Z  gs://asap-raw-data-team-sulzer/fastqs/nigra/OJ10_S1_L002_I2_001.fastq.gz#1701821943511357  metageneration=1
3410135611  2023-12-06T00:19:03Z  gs://asap-raw-data-team-sulzer/fastqs/nigra/OJ10_S1_L002_R1_001.fastq.gz#1701821943811147  metageneration=1
255816

In [352]:

!gsutil -u dnastack-asap-parkinsons mv "gs://asap-raw-data-team-sulzer/fastqs/nigra/**/*.fastq.gz" "gs://asap-raw-data-team-sulzer/fastqs/nigra/"


Copying gs://asap-raw-data-team-sulzer/fastqs/nigra/OJ1/fastq/SC_3PGEX_10X/220225_A00521_0295_AHMW3YDSX2/OJ1_S1_L001_I1_001.fastq.gz [Content-Type=application/octet-stream]...
Removing gs://asap-raw-data-team-sulzer/fastqs/nigra/OJ1/fastq/SC_3PGEX_10X/220225_A00521_0295_AHMW3YDSX2/OJ1_S1_L001_I1_001.fastq.gz...
Copying gs://asap-raw-data-team-sulzer/fastqs/nigra/OJ1/fastq/SC_3PGEX_10X/220225_A00521_0295_AHMW3YDSX2/OJ1_S1_L001_I2_001.fastq.gz [Content-Type=application/octet-stream]...
Removing gs://asap-raw-data-team-sulzer/fastqs/nigra/OJ1/fastq/SC_3PGEX_10X/220225_A00521_0295_AHMW3YDSX2/OJ1_S1_L001_I2_001.fastq.gz...
Copying gs://asap-raw-data-team-sulzer/fastqs/nigra/OJ1/fastq/SC_3PGEX_10X/220225_A00521_0295_AHMW3YDSX2/OJ1_S1_L001_R1_001.fastq.gz [Content-Type=application/octet-stream]...
Removing gs://asap-raw-data-team-sulzer/fastqs/nigra/OJ1/fastq/SC_3PGEX_10X/220225_A00521_0295_AHMW3YDSX2/OJ1_S1_L001_R1_001.fastq.gz...
Copying gs://asap-raw-data-team-sulzer/fastqs/nigra/OJ1/fastq

In [341]:
!gsutil -u dnastack-asap-parkinsons mv "gs://asap-raw-data-team-sulzer/fastqs/cingulate/bam_files" "gs://asap-raw-data-team-sulzer/artifacts/bam_files"


Copying gs://asap-raw-data-team-sulzer/fastqs/cingulate/bam_files/OF001/index.html [Content-Type=text/html]...
Removing gs://asap-raw-data-team-sulzer/fastqs/cingulate/bam_files/OF001/index.html...
Copying gs://asap-raw-data-team-sulzer/fastqs/cingulate/bam_files/OF001/possorted_genome_bam.bam [Content-Type=application/octet-stream]...
Removing gs://asap-raw-data-team-sulzer/fastqs/cingulate/bam_files/OF001/possorted_genome_bam.bam...
Copying gs://asap-raw-data-team-sulzer/fastqs/cingulate/bam_files/OF002/index.html [Content-Type=text/html]...
Removing gs://asap-raw-data-team-sulzer/fastqs/cingulate/bam_files/OF002/index.html...
Copying gs://asap-raw-data-team-sulzer/fastqs/cingulate/bam_files/OF002/possorted_genome_bam.bam [Content-Type=application/octet-stream]...
Removing gs://asap-raw-data-team-sulzer/fastqs/cingulate/bam_files/OF002/possorted_genome_bam.bam...

==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil 

In [335]:
!gsutil -u dnastack-asap-parkinsons mv "gs://asap-raw-data-team-sulzer/fastqs/**/analysis/*tar*" "gs://asap-raw-data-team-sulzer/artifacts/cellranger_counts"


Copying gs://asap-raw-data-team-sulzer/fastqs/cingulate/OF001/analysis/201123_OSAMA_FATIMA_2_HUMAN_10X-OF001-cellranger-count-default/OF001_cellranger_count_info.tar.gz [Content-Type=application/x-tar]...
Removing gs://asap-raw-data-team-sulzer/fastqs/cingulate/OF001/analysis/201123_OSAMA_FATIMA_2_HUMAN_10X-OF001-cellranger-count-default/OF001_cellranger_count_info.tar.gz...
Copying gs://asap-raw-data-team-sulzer/fastqs/cingulate/OF001/analysis/201123_OSAMA_FATIMA_2_HUMAN_10X-OF001-cellranger-count-default/OF001_cellranger_count_outs.tar [Content-Type=application/x-tar]...
Removing gs://asap-raw-data-team-sulzer/fastqs/cingulate/OF001/analysis/201123_OSAMA_FATIMA_2_HUMAN_10X-OF001-cellranger-count-default/OF001_cellranger_count_outs.tar...
Copying gs://asap-raw-data-team-sulzer/fastqs/cingulate/OF002/analysis/201123_OSAMA_FATIMA_2_HUMAN_10X-OF002-cellranger-count-default/OF002_cellranger_count_info.tar.gz [Content-Type=application/x-tar]...
Removing gs://asap-raw-data-team-sulzer/fastq

## get artifacts

 "gs://asap-raw-data-team-sulzer/manifest.txt"

In [90]:
!gsutil -u dnastack-asap-parkinsons -m cp -r \
  "gs://asap-raw-data-team-sulzer/artifacts" \
  /Users/ergonyc/Projects/ASAP/team-sulzer/

If you experience problems with multiprocessing on MacOS, they might be related to https://bugs.python.org/issue33725. You can disable multiprocessing by editing your .boto config or by adding the following flag to your command: `-o "GSUtil:parallel_process_count=1"`. Note that multithreading is still available even if you disable multiprocessing.

Copying gs://asap-raw-data-team-sulzer/artifacts/cingulate_lineages_sublineages_assigned_dims18_k20_res02_final.rds...
==> NOTE: You are downloading one or more large file(s), which would            
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

Copying gs://asap-raw-data-team-sulzer/artifacts/master_seurat_PD_filtered_doublet_removed_SCT_clusterslm3_res01_linege_called.rds...
Copying gs://asap-raw-data-team-sulzer/artifacts/nigra_cellbender.rds...        
Copying gs://asap-raw-data-team-sulzer/artifacts/nigra_obj

In [91]:
!gsutil -u dnastack-asap-parkinsons -m cp -r \
  "gs://asap-raw-data-team-sulzer/scripts" \
  /Users/ergonyc/Projects/ASAP/team-sulzer/

If you experience problems with multiprocessing on MacOS, they might be related to https://bugs.python.org/issue33725. You can disable multiprocessing by editing your .boto config or by adding the following flag to your command: `-o "GSUtil:parallel_process_count=1"`. Note that multithreading is still available even if you disable multiprocessing.

Copying gs://asap-raw-data-team-sulzer/scripts/cellbender_sample_processing.R...
/ [1/1 files][  1.9 KiB/  1.9 KiB] 100% Done                                    
Operation completed over 1 objects/1.9 KiB.                                      


In [ ]:
gsutil -m cp \
  "gs://asap-raw-data-team-sulzer/artifacts/cingulate_lineages_sublineages_assigned_dims18_k20_res02_final.rds" \
  "gs://asap-raw-data-team-sulzer/artifacts/master_seurat_PD_filtered_doublet_removed_SCT_clusterslm3_res01_linege_called.rds" \
  "gs://asap-raw-data-team-sulzer/artifacts/nigra_cellbender.rds" \
  "gs://asap-raw-data-team-sulzer/artifacts/nigra_obj_dims16_res02_k20_sct_harmony_dblrm_processed_final.rds" \
  .